# fMRI t-SNE maps, MD annotation

#### Damon Crockett, Software Studies, UCSD, damoncrockett@gmail.com

Some documentation here:

http://www.nitrc.org/plugins/mwiki/index.php/neurobureau:AthenaPipeline

This particular data, from the Pittsburgh study, has 89 subjects: "Skullstripped T1 weighted anatomical image in template space at 1mm x 1mm x 1mm voxel resolution"

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("./Pittsburgh_phenotypic.csv")

In [3]:
del df['Secondary Dx ']
del df['ADHD Index']
del df['ADHD Measure']
del df['Inattentive']
del df['Hyper/Impulsive']
del df['Med Status']
del df['QC_Rest_2']
del df['QC_Rest_3']
del df['QC_Rest_4']
del df['QC_Anatomical_2']
del df['IQ Measure']
del df['Site']
del df['DX']
del df['QC_Rest_1']
del df['QC_Anatomical_1']
del df['Verbal IQ']
del df['Performance IQ']
del df['Full2 IQ']

#### Pure Grid

In [4]:
from PIL import Image, ImageDraw

In [5]:
thumb = (192,227)

In [6]:
red = "#E55169"
blue = "#75D9FC" 
yellow = "#FDCC5B"
orange = "#FA8242"

In [7]:
from scipy.stats import percentileofscore as pct

In [8]:
df['age_pct'] = [pct(df['Age'],item,kind="weak") for item in df['Age']]
df['iq_pct'] = [pct(df['Full4 IQ'],item,kind="weak") for item in df['Full4 IQ']]

In [9]:
DIR = "/Users/damoncrockett/Desktop/fMRI/"

In [10]:
df['folder'] = [DIR + "wssd00" + str(item) + "/" for item in df['ScanDir ID']]

In [11]:
import numpy as np

In [12]:
def draw_hashes():
    x = np.arange(12,192,12)
    y1 = np.repeat(0,15)
    y2 = [6*item for item in [1,2,1,3,1,2,1,4,1,2,1,3,1,2,1]]
    
    y3 = np.repeat(227,15)
    y4 = [227 - item for item in y2]
    
    for i in range(15):
        draw.line([(x[i],y1[i]),(x[i],y2[i])],fill="white",width=1)
        draw.line([(x[i],y3[i]),(x[i],y4[i])],fill="white",width=1)

In [13]:
brain_slices = range(3,158)

In [14]:
from sklearn.manifold import TSNE as tsne
from skimage.io import imread
from shapely.geometry import Point
from skimage.color import rgb2gray

In [15]:
def check_duplicates(num_bins):
    x = list(pd.cut(subspace_tsne['x'],num_bins,labels=False))
    y = list(pd.cut(subspace_tsne['y'],num_bins,labels=False))
    tmp = pd.DataFrame(x)
    tmp['y'] = y
    n = np.mean(tmp.duplicated())
    return n

In [18]:
for brain_slice in brain_slices:   
    
    brain_slice = str(brain_slice).zfill(3)
    
    im_vectors = []
    local_paths = []
    
    for i in range(len(df)):
        im = df.folder.loc[0]+"image-slice"+brain_slice+".jpg"
        local_paths.append(im)
        im_vector = np.hstack(rgb2gray(imread(im)))
        im_vectors.append(im_vector)
    X = np.vstack(im_vectors)
    
    tsne_obj = tsne(n_components=2)
    subspace_tsne = pd.DataFrame(tsne_obj.fit_transform(X),columns=["x","y"])
    
    num_bins = 2
    while check_duplicates(num_bins) > 0.1:
        num_bins+=1
    
    subspace_tsne['grid_x'] = pd.cut(subspace_tsne['x'],num_bins,labels=False)
    subspace_tsne['grid_y'] = pd.cut(subspace_tsne['y'],num_bins,labels=False)
    subspace_tsne['local_path'] = local_paths
    
    grid_side = num_bins
    px_w = thumb[0] * grid_side
    px_h = thumb[1] * grid_side
    
    canvas = Image.new('RGB',(px_w,px_h),(0,0,0))
    
    for i in range(len(df)):
        
        folder = df.folder.loc[i]
        age = df.age_pct.loc[i] / 100
        iq = df.iq_pct.loc[i] / 100

        if df.Handedness.loc[i]==1:
            top_fill = red
        if df.Handedness.loc[i]==0:
            top_fill = yellow
        if df.Gender.loc[i]==1:
            bottom_fill = blue 
        if df.Gender.loc[i]==0:    
            bottom_fill = orange

        im = Image.open(folder+"image-slice"+brain_slice+".jpg")
        im.thumbnail(thumb,Image.ANTIALIAS)
        draw = ImageDraw.Draw(im)

        draw.line([(0,0),(int(round(thumb[0]*iq)),0)],fill=top_fill,width=10)
        draw.line([(0,225),(int(round(thumb[0]*age)),225)],fill=bottom_fill,width=10)
        draw_hashes()
    
        x = subspace_tsne.grid_x.loc[i] * thumb[0]
        y = subspace_tsne.grid_y.loc[i] * thumb[1]
        canvas.paste(im,(x,y))
    
    canvas.save(DIR+"maps/"+brain_slice+".png")